In [2]:
import faiss


In [4]:
!nvcc --version

import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.version.cuda)         # Should print 12.4 or the desired CUDA version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Fri_Jun_14_16:44:19_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.20
Build cuda_12.6.r12.6/compiler.34431801_0
True
12.6


In [1]:
!pip install datasets


Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   ------------------------------------- -- 23.6/25.3 MB 114.9 MB/s eta 0:00:01
   ---------------------------------------- 25.3/25.3 MB 106.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import AutoTokenizer, RagRetriever, RagModel, RagSequenceForGeneration, RagTokenizer


C:\Users\Chloe\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
from datasets import load_dataset

# Load WikiText-2 dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
print(dataset)  # Check the structure (should have 'text' column)

c:\Program Files\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chloe\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 2281293.66 examples/s]

Dataset({
    features: ['text'],
    num_rows: 36718
})


In [7]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch

encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Function to create embeddings
def create_embeddings(dataset):
    embeddings = []
    for doc in dataset:
        # Tokenize the text
        inputs = tokenizer(doc['text'], truncation=True, padding=True, return_tensors="pt", max_length=512)

        # Generate embeddings
        with torch.no_grad():
            output = encoder(**inputs)
            embeddings.append(output.pooler_output.squeeze().cpu().numpy())
    return embeddings

# Create embeddings for the WikiText dataset
embeddings = create_embeddings(dataset)

c:\Program Files\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chloe\.cache\huggingface\hub\models--facebook--dpr-ctx_encoder-single-nq-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when

In [8]:
import faiss
import numpy as np

# Convert embeddings into a numpy array
embedding_matrix = np.array(embeddings)

# Create FAISS index using L2 distance
index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # L2 distance
index.add(embedding_matrix)  # Add the embeddings to the index

# Save the FAISS index for later use
faiss.write_index(index, "wikitext_index.faiss")

In [9]:
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
# retriever = RagRetriever.from_pretrained(
#     "facebook/rag-token-base", index_name="exact"
# )
retriever = RagRetriever.from_pretrained(
    "facebook/rag-token-base",
    index_name=index,  # Custom index name
    # passages_path="path/to/your/custom_dataset",  # Optional: path to dataset (not needed if you have a FAISS index)
    index_path="wikitext_index.faiss",  # Path to the FAISS index file
)

c:\Program Files\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chloe\.cache\huggingface\hub\models--facebook--rag-token-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. 

TypeError: Object of type IndexFlatL2 is not JSON serializable